# PuLP Optimization Example: Bin Assignment for Warehouse Storage

Scenario:
We have 10 materials and 5 storage bins. Each material has:
- A frequency score (1 = low, 3 = high)
- A size (volume in units)
- A tag (e.g., fragile, hazardous, regular)

Each bin has:
- A capacity limit (volume)
- A distance score from the dispatch zone (lower is better)
- Bin type (regular, safe, special)

Goal:
Assign materials to bins minimizing picking distance for high-frequency items,
while respecting size limits and compatibility constraints (e.g., fragile items can only go to 'safe' bins).

In [2]:
import pandas as pd
from pulp import LpProblem, LpMinimize, LpVariable, lpSum, LpBinary, value

# Step 1: Define the data
materials = pd.DataFrame({
    'material': [f'M{i}' for i in range(1, 11)],
    'frequency': [3, 1, 2, 3, 2, 1, 3, 2, 1, 3],
    'size': [5, 4, 6, 7, 3, 2, 6, 5, 4, 6],
    'tag': ['fragile', 'regular', 'hazardous', 'regular', 'regular',
            'hazardous', 'fragile', 'regular', 'hazardous', 'fragile']
})

bins = pd.DataFrame({
    'bin': [f'B{i}' for i in range(1, 6)],
    'capacity': [15, 20, 10, 25, 12],
    'distance': [1, 2, 3, 1, 4],
    'type': ['safe', 'regular', 'special', 'regular', 'safe']
})

# Step 2: Build compatibility matrix (1 if compatible, 0 if not)
compatibility = {}
for _, item in materials.iterrows():
    for _, b in bins.iterrows():
        if item['tag'] == 'fragile' and b['type'] != 'safe':
            compatibility[(item['material'], b['bin'])] = 0
        elif item['tag'] == 'hazardous' and b['type'] != 'special':
            compatibility[(item['material'], b['bin'])] = 0
        else:
            compatibility[(item['material'], b['bin'])] = 1

# Step 3: Define the problem
model = LpProblem("BinAssignmentOptimization", LpMinimize)

# Decision variables
x = LpVariable.dicts("assign", (materials['material'], bins['bin']), cat=LpBinary)

# Objective: minimize frequency-weighted distance
model += lpSum(x[i][b] * materials.set_index('material').loc[i, 'frequency'] * 
               bins.set_index('bin').loc[b, 'distance']
               for i in materials['material'] for b in bins['bin'])

# Constraint: each material assigned to one bin
for i in materials['material']:
    model += lpSum(x[i][b] for b in bins['bin']) == 1

# Constraint: bin capacity
for b in bins['bin']:
    model += lpSum(x[i][b] * materials.set_index('material').loc[i, 'size']
                   for i in materials['material']) <= bins.set_index('bin').loc[b, 'capacity']

# Constraint: respect compatibility
for i in materials['material']:
    for b in bins['bin']:
        model += x[i][b] <= compatibility[(i, b)]

# Solve
model.solve()

# Output results
print("\nAssigned Material -> Bin")
for i in materials['material']:
    for b in bins['bin']:
        if x[i][b].varValue == 1:
            print(f"{i} -> {b}")

# Show objective value
print("\nTotal Weighted Distance:", value(model.objective))



Assigned Material -> Bin
M1 -> B1
M2 -> B4
M3 -> B3
M4 -> B4
M5 -> B4
M6 -> B3
M8 -> B4
M10 -> B1

Total Weighted Distance: 34.000000029999995
